In [197]:
import pandas as pd
import numpy as np
import re #Regex lib
from forex_python.converter import CurrencyRates #currencies lib
from datetime import datetime
import matplotlib
import json


### Read in the dataframes and create a dict {(year, dataframe)}


In [198]:
# Create a list of filenames for the datasets
years = [2017, 2018, 2019, 2020]
file_names = ['datasets/{}.csv'.format(year) for year in years]

# Read in the dataframes and create a dict {(year, dataframe)}
data_frames_dict = {}

#data_frames_dict[2017] = pd.read_csv('datasets/2017.csv', encoding = "ISO-8859-1").filter(items=['Country', 'Age']).dropna(thresh=2) 
data_frames_dict[2018] = pd.read_csv('datasets/2018.csv', encoding = "ISO-8859-1").filter(items=['Country',  'Age']).dropna(thresh=2) 
data_frames_dict[2019] = pd.read_csv('datasets/2019.csv', encoding = "ISO-8859-1").filter (items=['Country',  'Age']).dropna(thresh=2) 
data_frames_dict[2020] = pd.read_csv('datasets/2020.csv', encoding = "ISO-8859-1").filter (items=['Country',  'Age']).dropna(thresh=2) 
#preprocess 2021
def check_countryType(country, US_state, UK_country):
    
    if  isinstance(US_state, str) or country == 'United States':
        return 'United States of America'
    elif isinstance(UK_country, str) or country == 'United Kingdom of Great Britain and Northern Ireland':
        return 'United Kingdom'
    else:
        return country


df_2021 = pd.read_csv('datasets/2021.csv', encoding = "ISO-8859-1").filter (items=['Country', 'US_State', 'UK_Country',  'Age'])
df_2021 = df_2021.dropna(subset = [ 'Age'])
df_2021.Country =  df_2021.apply(lambda row : check_countryType(row['Country'],row['US_State'],  
                     row['UK_Country']), axis=1)
df_2021 = df_2021.drop(['US_State', 'UK_Country'], axis = 1).dropna(thresh=2)
data_frames_dict[2021]  = df_2021
df_2021

/usr/local/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Country,Age
0,Slovakia,25-34 years old
1,Netherlands,18-24 years old
2,Russian Federation,18-24 years old
3,Austria,35-44 years old
4,United Kingdom,25-34 years old
...,...,...
83434,United States of America,25-34 years old
83435,Benin,18-24 years old
83436,United States of America,25-34 years old
83437,Canada,25-34 years old


In [199]:
def age_mapper(age):
    if age < 18:
        return 'Under 18 years old'
    elif age >= 18 and  age<=24:
        return '18 - 24 years old'
    elif age >= 25 and age <= 34:
        return '25 - 34 years old'
    elif age >= 35 and age <=  44:
        return '35 - 44 years old'
    elif age >= 45 and age <= 54: 
        return '45 - 54 years old' 
    elif age >= 55  and age <= 64:
        return '55 - 64 years old'
    elif age >= 65:
        return '65 years or older'
    else:
        raise Exception("Age invalid")
        

### obtain developer types 


In [200]:
import codecs

for year, df in data_frames_dict.items():
    if year != 2018 and year != 2021  :
        df.Age = df.Age.apply(lambda x: age_mapper(int(x)))
    df['AgeCount'] = pd.Series(np.ones(len(df.Age), dtype=np.int32), index=df.index)  
    df = df.groupby(by=['Country','Age'] , as_index=False).sum()

    json_dict = {}
    for k, v in df.groupby('Country'):
        inner_dict = {}
        for age, count in zip(v['Age'], v['AgeCount']):
            inner_dict[age] = count 
        json_dict[k] = inner_dict
        
        
    with codecs.open("processed_data/age/{}-age_count_per_country.json".format(year), "w", encoding='utf-8') as outfile:
        json.dump(json_dict, outfile, indent = 4, ensure_ascii=False)


        


        


In [201]:
json_dict


{'Afghanistan': {'18-24 years old': 14,
  '25-34 years old': 15,
  '35-44 years old': 3,
  '45-54 years old': 1,
  '55-64 years old': 2,
  '65 years or older': 2,
  'Prefer not to say': 6,
  'Under 18 years old': 13},
 'Albania': {'18-24 years old': 34,
  '25-34 years old': 20,
  '35-44 years old': 7,
  '45-54 years old': 1,
  '65 years or older': 1,
  'Under 18 years old': 6},
 'Algeria': {'18-24 years old': 15,
  '25-34 years old': 24,
  '35-44 years old': 2,
  '45-54 years old': 2,
  '55-64 years old': 1,
  'Under 18 years old': 1},
 'Andorra': {'18-24 years old': 2,
  '25-34 years old': 5,
  '35-44 years old': 1,
  '45-54 years old': 1,
  'Under 18 years old': 1},
 'Angola': {'18-24 years old': 11,
  '25-34 years old': 8,
  '35-44 years old': 3,
  '65 years or older': 3,
  'Under 18 years old': 1},
 'Argentina': {'18-24 years old': 138,
  '25-34 years old': 218,
  '35-44 years old': 146,
  '45-54 years old': 49,
  '55-64 years old': 7,
  'Prefer not to say': 2,
  'Under 18 years ol